### Dataframes

In [2]:
from collections import Counter
import pandas as pd
import os
import datetime

In [3]:
df_tickets = pd.read_excel(f"DTICKETHISTAB_BETON.xlsx",engine='openpyxl')

In [3]:
df_orders = pd.read_excel(f"DORDERSTAB_BETON.xlsx",sheet_name='data',engine='openpyxl')

### Matrice de temps

In [4]:
df_temps = pd.read_csv(f"Matrice_de_temps_all.csv",header=None)

In [5]:
temps = df_temps.to_numpy()

In [6]:
df_dist = pd.read_csv(f"Matrice_de_distance_all.csv",header=None)
dist = df_dist.to_numpy()

### Localisation des dépots

In [7]:
df_depots = pd.read_excel(f"adresses_complete.xlsx",engine='openpyxl')
df_depots = df_depots[df_depots['Location_type'] != 'customer']
df_depots.dropna(axis=1,  inplace=True)
df_depots['NBR'] = df_depots['JOB_DESC'].apply(
    lambda x: x if len(x.split(' ')) == 1 else int(x.split(' ')[1]))
df_depots.set_index('NBR',inplace=True)
df_depots.drop(['Location_type', 'JOB_DESC', 'MAP_PAGE', 
       'Postal_code', 'JOBSTATE', 'LATITUDE_Y', 'LONGITUDE_X'],axis=1,inplace=True)

In [8]:
def beginTime(week,day,driver):
    week_op = df_tickets[df_tickets['WEEK']==week]
    day_op = week_op[week_op['DATE']==day]
    _data = day_op[day_op['DRIVER_NBR']==driver][["BEGIN_LOAD","ARRIVE_PLANT"]]
    _data.sort_values(["BEGIN_LOAD"],inplace=True)
   
    return f"{_data.iloc[0,0].time()}".split('.')[0]

def endTime(week,day,driver):
    week_op = df_tickets[df_tickets['WEEK']==week]
    day_op = week_op[week_op['DATE']==day]
    _data = day_op[day_op['DRIVER_NBR']==driver][["BEGIN_LOAD","ARRIVE_PLANT"]]
    _data.sort_values(["BEGIN_LOAD"],inplace=True)
    return f"{_data.iloc[-1,1].time()}".split('.')[0]

In [49]:
df_tickets.dropna(axis=0,inplace=True)
df_tickets['CUST'] = df_tickets['ORDER_ID'].apply(lambda x: df_orders[df_orders['ORDER_ID']==x]['CUST_NBR'].max())
df_tickets['JOB_DESC'] = df_tickets['ORDER_ID'].apply(lambda x: df_orders[df_orders['ORDER_ID']==x]['JOB_DESC'].max())
    
df_tickets['TDATE'] = df_tickets['TICKET_DATE'].apply(lambda x:datetime.date(x.year,x.month,x.day))
df_tickets['WEEK'] = df_tickets['TDATE'].apply(lambda x:datetime.date(x.year,x.month,x.day).isocalendar()[1])
df_tickets['DAY'] = df_tickets['TDATE'].apply(lambda x:datetime.date(x.year,x.month,x.day).toordinal())

df_tickets = df_tickets[df_tickets['SHIP_LOC']!=99]
df_tickets = df_tickets[df_tickets['JOB_DESC'] != 'PICKUP']

df_tickets.drop(['DTICKETHIS_TICKET_ID',  'TICKET_NBR', 'TICKET_DATE',
       'TICKET_TIME',   'BEGIN_POUR', 'FINISH_POUR',   'FINISH_LOAD',
       'TO_PLANT', 'ON_JOB', 'TO_JOB', 'LOAD', 'LOADNR',
       'DTICKETPRODUCTHIS_TICKET_ID',   'CATEGORY_CODE'],axis=1,inplace=True)


In [56]:
df_orders = df_orders[df_orders['JOB_DESC']!='PICKUP']
df_orders['DATE'] = df_orders['SHIPDATE'].apply(lambda x:datetime.date(x.year,x.month,x.day))

In [57]:
df_tickets['DATE'] = df_tickets.apply(lambda x:  df_orders[df_orders['ORDER_ID']==x.ORDER_ID]['DATE'].min(),axis=1)

In [59]:
df_orders.drop(['COMP_NBR',  'ORDER_STATUS',
       'ORDER_DATE', 'ORDER_TIME (min)', 'SHIPDATE', 'Day_of_the_week',
       'SHIPTIME (min)', 'ZONE_NBR', 'WORK_TYPE', 'SCHED_LOC',
       'JOB_DESC', 'ORDER_DESC', 'MAP_PAGE', 'JOBCITY', 'Postal code ',
       'JOBSTATE', 'LATITUDE_Y', 'LONGITUDE_X', 'TRAVEL_MINUTES',
       'UNLOAD_MINUTES', 'QTY_PER_HOUR', 'TRUCK_REQUEST',
       'CATEGORY_CODE', 'UNIT_OF_MEASURE', 'QUANTITY', 'LOOKUP_DESC',
       'CONFIG_DESC', 'Unnamed: 33', 'Unnamed: 34', 'Location ',
       'ID','DATE'],axis=1,inplace=True)

In [60]:
# df_tickets.reset_index()
df=pd.merge(left=df_tickets,right= df_orders,how="inner",on=['ORDER_ID','PROD_NBR'])
df.head()

,SHIP_LOC,ORDER_ID,DRIVER_NBR,BEGIN_LOAD,ARRIVE_PLANT,TRUCK_NBR,SCHED_LOC,PROD_NBR,QUANTITY,DATE,CUST,JOB_DESC,TDATE,WEEK,DAY,ORDER_LOC,ORDER_NBR,CUST_NBR,LOCATION ID
0,54,917231,15083.0,2020-10-01 14:16:23.720,2020-10-01 16:34:16.832,460,54,3510RPC1,8.0,2020-10-01,104590,261 RUE QUEEN (ET KING),2020-10-01,40,737699,0,603592,104590,0
1,54,917231,15143.0,2020-10-01 15:00:56.091,2020-10-01 16:34:14.566,370,54,3510RPC1,8.0,2020-10-01,104590,261 RUE QUEEN (ET KING),2020-10-01,40,737699,0,603592,104590,0
2,54,917231,15070.0,2020-10-01 15:20:11.886,2020-10-01 17:10:50.250,426,54,3510RPC1,8.0,2020-10-01,104590,261 RUE QUEEN (ET KING),2020-10-01,40,737699,0,603592,104590,0
3,54,917231,15507.0,2020-10-01 15:59:26.468,2020-10-01 18:21:08.017,384,54,3510RPC1,8.0,2020-10-01,104590,261 RUE QUEEN (ET KING),2020-10-01,40,737699,0,603592,104590,0
4,54,917231,7000059.0,2020-10-01 16:15:50.000,2020-10-01 18:35:24.000,019,54,3510RPC1,7.0,2020-10-01,104590,261 RUE QUEEN (ET KING),2020-10-01,40,737699,0,603592,104590,0


In [61]:
df['SHIP_LOC_ID'] = df.apply(lambda x: df_depots.loc[x.SHIP_LOC]['Location ID'],axis=1)

In [62]:
df['SCHED_LOC_ID'] = df.apply(lambda x: df_depots.loc[x.SCHED_LOC]['Location ID'],axis=1)

In [63]:
df['t1']=df.apply(lambda x: (temps[x.SHIP_LOC_ID][x['LOCATION ID']]+temps[x['LOCATION ID']][x.SHIP_LOC_ID])/60,axis=1)

In [64]:
df['t2']=df.apply(lambda x: (temps[x.SCHED_LOC_ID][x['LOCATION ID']]+temps[x['LOCATION ID']][x.SCHED_LOC_ID])/60,axis=1)

In [65]:
df['d1']=df.apply(lambda x: (dist[x.SHIP_LOC_ID][x['LOCATION ID']]+dist[x['LOCATION ID']][x.SHIP_LOC_ID]),axis=1)

In [66]:
df['d2']=df.apply(lambda x: (dist[x.SCHED_LOC_ID][x['LOCATION ID']]+dist[x['LOCATION ID']][x.SCHED_LOC_ID]),axis=1)

In [73]:
df.reset_index(inplace=True)
df.set_index(['DATE'],inplace=True)
df.sort_index(inplace=True)
df.groupby(by=['DATE']).agg({'QUANTITY':'sum','t1':'sum','t2':'sum' 
    ,'d1':'sum','d2':'sum','DRIVER_NBR':'nunique','TRUCK_NBR':'nunique'}).to_csv('stat_by_day.csv',
                                                                        sep=';',decimal=',')